In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/rwanda/Test.csv
/kaggle/input/rwanda/VariableDefinitions.csv
/kaggle/input/rwanda/Train.csv
/kaggle/input/rwanda/SampleSubmission.csv
/kaggle/input/rwanda/Riders.csv


In [2]:
train=pd.read_csv('/kaggle/input/rwanda/Train.csv')
test=pd.read_csv('/kaggle/input/rwanda/Test.csv')
rid=pd.read_csv('/kaggle/input/rwanda/Riders.csv')
sub=pd.read_csv('/kaggle/input/rwanda/SampleSubmission.csv')

In [3]:
len(test.columns)

25

In [4]:
train.describe()

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival
count,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,16835.000000,552.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000
mean,2.752182,15.653696,3.240083,15.653837,3.240225,15.653837,3.240225,15.653837,3.240225,15.653837,3.240225,9.506533,23.258889,7.905797,-1.281470,36.811264,-1.282581,36.811220,1556.920947
std,0.625178,8.798916,1.567295,8.798886,1.567228,8.798886,1.567228,8.798886,1.567228,8.798886,1.567228,5.668963,3.615768,17.089971,0.030507,0.037473,0.034824,0.044721,987.270788
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.200000,0.100000,-1.438302,36.653621,-1.430298,36.606594,1.000000
25%,3.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,5.000000,20.600000,1.075000,-1.300921,36.784605,-1.301201,36.785661,882.000000
50%,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,8.000000,23.500000,2.900000,-1.279395,36.807040,-1.284382,36.808002,1369.000000
75%,3.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,13.000000,26.000000,4.900000,-1.257147,36.829741,-1.261177,36.829477,2040.000000
max,4.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,49.000000,32.100000,99.100000,-1.147170,36.991046,-1.030225,37.016779,7883.000000


In [ ]:
test.describe()

# CLEANING DATA

In [5]:
l=[ i for i in train.columns if i not in test.columns]
l

['Arrival at Destination - Day of Month',
 'Arrival at Destination - Weekday (Mo = 1)',
 'Arrival at Destination - Time',
 'Time from Pickup to Arrival']

In [6]:
train.drop(['Arrival at Destination - Day of Month',
 'Arrival at Destination - Weekday (Mo = 1)',
 'Arrival at Destination - Time'],axis=1,inplace = True)

# GENERATING DATETIME FEATURES

In [7]:
time=[i for i in train.columns if 'Time' in i]
time=['Placement - Time',
 'Confirmation - Time',
 'Arrival at Pickup - Time',
 'Pickup - Time']

In [8]:
for i in time:    
    train[i] = pd.to_datetime(train[i])
    train[i+'-hour'] = train[i].dt.hour
    train[i+'-minute'] = train[i].dt.minute


In [9]:
for i in time:    
    test[i] = pd.to_datetime(test[i])
    test[i+'-hour'] = test[i].dt.hour
    test[i+'-minute'] = test[i].dt.minute

In [10]:
data=pd.concat([train,test])
len(data.columns)

34

In [11]:
selected = list(set(['Order No', 'User Id', 'Platform Type',
       'Personal or Business', 'Placement - Day of Month',
       'Placement - Weekday (Mo = 1)', 'Confirmation - Day of Month',
       'Confirmation - Weekday (Mo = 1)', 'Arrival at Pickup - Day of Month',
       
       'Distance (KM)', 'Temperature',
        'Pickup Lat', 'Pickup Long',
       'Destination Lat', 'Destination Long', 'Rider Id',
       'Placement - Time-hour',
       'Placement - Time-minute', 'Confirmation - Time-hour',
       'Confirmation - Time-minute', 'Arrival at Pickup - Time-hour',
       'Arrival at Pickup - Time-minute', 'Pickup - Time-hour',
       'Pickup - Time-minute','Time from Pickup to Arrival','Placement - Time',
 'Confirmation - Time',
 'Arrival at Pickup - Time',
 'Confirmation - Time','Pickup - Time']))
train=data[selected]

# FEATURE ENGINEERING

-TIME FROM PLACEMENT TO CONFIRMATION 

In [12]:
train['time_placem_confirm'] = train['Confirmation - Time'] - train['Placement - Time']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


-TIME FROM ARRIVAL TO PICKUP TO PICKUP

In [13]:
train['time_Arvpick_pick'] = train['Pickup - Time'] - train['Arrival at Pickup - Time']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
cl=['time_placem_confirm','time_Arvpick_pick']
for i in cl:    
    train[i+'-minute'] = train[i].dt.components['minutes']
    train[i+'-sec'] = train[i].dt.components['seconds']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


CONVERSION TO SECONDS

In [15]:
train['time_placem_confirm'] = train['time_placem_confirm-minute']*60 + train['time_placem_confirm-sec']
train['time_Arvpick_pick'] = train['time_Arvpick_pick-minute']*60 + train['time_Arvpick_pick-sec']

ADDING THE RIDER METRICS

In [16]:
train=pd.merge(train,rid,on='Rider Id',how='left')

In [17]:
rid.describe()

,No_Of_Orders,Age,Average_Rating,No_of_Ratings
count,960.000000,960.000000,960.000000,960.000000
mean,1864.851042,1200.234375,13.412604,427.983333
std,1880.337785,810.930171,2.675794,486.957931
min,2.000000,96.000000,0.000000,0.000000
25%,261.750000,478.250000,13.500000,30.000000
50%,1475.500000,1021.000000,14.000000,223.000000
75%,2847.250000,1891.500000,14.300000,678.750000
max,9756.000000,3764.000000,15.200000,2298.000000


In [18]:
train['orders-per-day'] = train['No_Of_Orders'] / train['Age']

MINIMAL DISTANCE THAT THE RIDER CAN CROSS FROM PICKUP TO DESTINATION

In [19]:
!pip install vincenty

  Created wheel for vincenty: filename=vincenty-0.1.4-py3-none-any.whl size=3079 sha256=3a6aa1511a674c4f531880fa3726878a17e938737f7a662d5d8c5dd0b3cce793
  Stored in directory: /root/.cache/pip/wheels/2f/4d/df/6e2ce31c63c93508b38ed9098c1483e7616b55a6ddabcd2f5b
Successfully built vincenty


In [20]:
from vincenty import vincenty
d=[]
for i in range(len(train)):
    d.append(vincenty((train['Pickup Lat'][i],train['Pickup Long'][i]),(train['Destination Lat'][i],train['Destination Long'][i]) ))
train['mindistance'] =d

In [21]:
train['RF'] = (train['Average_Rating'] * train['No_of_Ratings']) / (train['Age'] )
train['PR'] = train['No_Of_Orders'] / train['Age']

In [22]:
train['Distance (KM)'] *= 1000
train['Distance (KM)'] = train['Distance (KM)'] / train['Distance (KM)'].max()

NUMBER OF RIDERS GOING TO THE SAME DESTINATION

In [23]:
v=train.groupby(['Destination Lat', 'Destination Long'])['Rider Id'].count().reset_index(name='rider destin to same place')
train=pd.merge(train,v,how='left',on=['Destination Lat', 'Destination Long'])

NUMBER OF RIDERS GOING TO THE SAME CUSTOMER TYPE USING THE SAME PLATFORM

In [24]:
v=train.groupby(['Platform Type', 'Personal or Business'])['Rider Id'].count().reset_index(name='rider type of the tawsila')
train=pd.merge(train,v,how='left',on=['Personal or Business', 'Platform Type'])

GENERATING SOME OTHER FEATURES

In [25]:
v=train.groupby(['User Id'])['Rider Id'].count().reset_index(name='User Type')
train=pd.merge(train,v,how='left',on=['User Id'])

In [26]:
v=train.groupby(['Average_Rating'])['Rider Id'].count().reset_index(name='Rider Type')
train=pd.merge(train,v,how='left',on=['Average_Rating'])

In [28]:
columnstr=['Order No', 'User Id', 'Platform Type','User Type','Rider Type',
           'Personal or Business',
           #'Placement - Day of Month',
           #'Placement - Weekday (Mo = 1)', 'Confirmation - Day of Month',
           #'Confirmation - Weekday (Mo = 1)',
           #'Arrival at Pickup - Day of Month',
           'Distance (KM)', 'Temperature',
           'Pickup Lat', 'Pickup Long',
           'Destination Lat', 'Destination Long', 'Rider Id',
           'Placement - Time-hour',
           'Placement - Time-minute', 'Confirmation - Time-hour',
           'Confirmation - Time-minute', 'Arrival at Pickup - Time-hour',
           'Arrival at Pickup - Time-minute', 'Pickup - Time-hour',
           'Pickup - Time-minute','time_placem_confirm','time_Arvpick_pick','No_Of_Orders', 'Age', 'Average_Rating', 'No_of_Ratings',
           'orders-per-day',
           'mindistance', 
           'RF',
           'PR',
           'rider destin to same place',
           'rider type of the tawsila'
]

ENCODING CATEGORICAL FEATURES

In [ ]:
from sklearn.preprocessing import LabelEncoder
for col in train.columns:
    if train[col].dtypes == 'object':
        label_encoder =LabelEncoder() 
        train[col]= label_encoder.fit_transform(train[col]) 
        

In [ ]:
train1=train.iloc[:21201,:]
test1=train.iloc[21201:,:]

# MODEL

REMEMBER TO ALLOW THE---''*GPU*''---ON YOUR KERNEL 

In [ ]:
from catboost import Pool, CatBoostRegressor
model = CatBoostRegressor(loss_function="RMSE",
                           eval_metric="RMSE",
                          task_type="GPU",
                           learning_rate=0.015,
                           iterations=15000,
                           l2_leaf_reg=100,
                           random_seed=42,
                           od_type="Iter",
                           depth=7,
                           early_stopping_rounds=3000,
                           border_count=32
                          )
catfeat=['User Id', 'Platform Type',
         'Personal or Business',
         'Rider Id',
         'rider type of the tawsila',
         'rider destin to same place' ,'User Type' ,'Rider Type']

model.fit(
    train1[columnstr], train1['Time from Pickup to Arrival'],
    #eval_set=(train1[columnstr], train1['Time from Pickup to Arrival']),
    cat_features=catfeat,
    logging_level='Silent',
    plot=True)

In [ ]:
pred=model.predict(test1[columnstr])
pred

In [ ]:
min(pred)

In [ ]:
#Submission
submit = pd.DataFrame({'Order_No': sub['Order_No'], 'Time from Pickup to Arrival': np.clip(pred,a_min=100,a_max=5000)})
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+"704+k.csv" , index=False)
    return FileLink(submission_name+"704+k.csv")
create_submission(submit, 'submit')

# FEATURE IMPORTANCE

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
feature_score = pd.DataFrame(list(zip(train1[columnstr].dtypes.index, model.get_feature_importance(Pool(train1[columnstr] , label=train1['Time from Pickup to Arrival'], cat_features=catfeat)))),
                columns=['Feature','Score'])

feature_score = feature_score.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

In [ ]:
plt.rcParams["figure.figsize"] = (9,7)
ax = feature_score.plot('Feature', 'Score', kind='bar', color='c')
ax.set_title("Catboost Feature Importance Ranking", fontsize = 14)
ax.set_xlabel('')

rects = ax.patches

labels = feature_score['Score'].round(2)

for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 0.35, label, ha='center', va='bottom')

plt.show()